In [1]:
import napari
import numpy as np
import matplotlib.pyplot as plt
import z5py
from pathlib import Path

from cryofib.n5_utils import read_volume, print_key_tree

## View n5 datasets of platynereis membrane segmentation

In [2]:
data_dir = Path("/g/kreshuk/buglakova/data/platynereis_em_membranes/membrane")
n5_paths = list(data_dir.glob("*.n5"))
f_n5_list = [z5py.File(n5_path, "r") for n5_path in n5_paths]

In [3]:
print(n5_paths)

[PosixPath('/g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_01.n5'), PosixPath('/g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_02.n5'), PosixPath('/g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_03.n5'), PosixPath('/g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_04.n5'), PosixPath('/g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_05.n5'), PosixPath('/g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_06.n5'), PosixPath('/g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_07.n5'), PosixPath('/g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_08.n5'), PosixPath('/g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_09.n5')]


In [4]:
print_key_tree(f_n5_list[0])

Key structure of z5 file /g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_01.n5
volumes
volumes/labels
volumes/labels/segmentation
volumes/labels/segmentation/s1
volumes/raw
volumes/raw/s0
volumes/raw/s1
volumes/raw/s2
volumes/raw/s3


## Read volumes

In [5]:
idx = 8
roi = np.s_[:]
raw_stacks = []
for s in range(4):
    raw_stacks.append(read_volume(f_n5_list[idx], ("volumes/raw/s%d"%s), roi))

segm = read_volume(f_n5_list[idx], "volumes/labels/segmentation/s1", roi)

No key volumes/raw/s0 in file /g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_09.n5
Key structure of z5 file /g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_09.n5
volumes
volumes/labels
volumes/labels/segmentation
volumes/labels/segmentation/s1
volumes/raw
volumes/raw/s1
Reading roi slice(None, None, None) of volume volumes/raw/s1 from /g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_09.n5
Read volume with shape (120, 1280, 1280), data type uint8
No key volumes/raw/s2 in file /g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_09.n5
Key structure of z5 file /g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_09.n5
volumes
volumes/labels
volumes/labels/segmentation
volumes/labels/segmentation/s1
volumes/raw
volumes/raw/s1
No key volumes/raw/s3 in file /g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_09.n5
Ke

## Open it in napari

In [6]:
v = napari.Viewer()

### Open untransformed fluorescence

In [7]:
resolution = [25, 20, 20]
# for s in range(4):
#     v.add_image(raw_stacks[s], name=("s%d"%s), scale=resolution)
v.add_image(raw_stacks[1], name="s1", scale=resolution)
v.add_labels(segm, name="segmentation", scale=resolution)

<Labels layer 'segmentation' at 0x7f5ef0321cc0>

In [8]:
from skimage.segmentation import find_boundaries

In [10]:
boundaries = find_boundaries(segm, connectivity=segm.ndim, mode="thick")

In [11]:
v.add_labels(boundaries, name="boundaries", scale=resolution)

<Labels layer 'boundaries' at 0x7f5ee041fdf0>

In [12]:
boundaries_inner = find_boundaries(segm, connectivity=segm.ndim, mode="inner")

In [14]:
v.add_labels(boundaries_inner, name="boundaries_inner", scale=resolution)

<Labels layer 'boundaries_inner [1]' at 0x7f5ed1da3760>

In [15]:
boundaries_outer = find_boundaries(segm, connectivity=segm.ndim, mode="outer")

In [16]:
v.add_labels(boundaries_outer, name="boundaries_outer", scale=resolution)

<Labels layer 'boundaries_outer' at 0x7f5ed1c52380>

In [17]:
foreground = (segm > 0).astype(int)

In [18]:
from skimage.morphology import binary_erosion

In [19]:
foreground = binary_erosion(foreground)

In [20]:
v.add_labels(foreground, name="foreground_erosion", scale=resolution)

<Labels layer 'foreground_erosion' at 0x7f5ed00bc7f0>

In [21]:
f_n5_list[0].